# Part 2 - Refine Data

The second step for analyzing the data is to perform some additional preparations and enrichments. While the first step of storing the data into the structured zone should be mainly a technical conversion without losing any information, this next step will integrate some data and also preaggregate weather data to simplify working with it.

In [ ]:
dwh_basedir = "/user/hadoop/nyc-dwh"
structured_basedir = dwh_basedir + "/structured"
refined_basedir = dwh_basedir + "/refined"

# 0 Prepare Python Environment

## 0.1 Spark Session

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

if not 'spark' in locals():
    spark = SparkSession.builder \
        .master("local[*]") \
        .config("spark.driver.memory","64G") \
        .getOrCreate()
    
spark.version

## 0.2 Matplotlib

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# 1 Read Taxi Data

Now we can read in the taxi data from the structured zone.

## 1.1 Trip Data

In [ ]:
trip_data = # YOUR CODE HERE
trip_data.limit(10).toPandas()

Just to be sure, let us inspect the schema. It should match exactly the specified one.

In [ ]:
trip_data.printSchema()

## 1.2 Fare information

Now we read in the second table containing the trips fare information.

In [ ]:
fare_data = spark.read.parquet(structured_basedir + "/taxi-fare")
fare_data.limit(10).toPandas()

In [ ]:
fare_data.printSchema()

## 1.3 Join datasets

We can now join both the trip information and the fare information together in order to get a complete picture. Since the trip records do not contain a technical unique key, we use the following columns as the composite primary key of each trip:
* medallion
* hack_license
* vendor_id
* pickup_datetime

Finally the result is stored into the refined zone into the subdirectory `taxi-trip`.

In [ ]:
taxi_trips = # YOUR CODE HERE


### Read from Refined Zone

In [ ]:
taxi_trips = spark.read.parquet(refined_basedir + "/taxi-trip")
taxi_trips.limit(10).toPandas()

In [ ]:
taxi_trips.printSchema()

In [ ]:
taxi_trips.count()

# 2. Weather Data

The weather data also requires some additional preprocessing, especially when we want to join against weather data. The primary problem of all measurements is, that they might happen at different time intervals and not all measurements contain all metrics. Therefore we preaggregate the weather data to hourly and daily measurements, which can directly be used for joining.

## 2.1 Weather Data

We already have weather data, but only individual measurements. We do not know how many measurements there are per hour and per day, so the raw table is not very useable for joining. Instead we'd like to have an hourly and a daily weather table containing average temperature, wind speed and precipitation. Since we are only interested in the year 2013, we also only load that specific year.

In [ ]:
weather = spark.read.parquet(structured_basedir + "/weather/2013")
weather.limit(10).toPandas()

## 2.2 Calculate derived metrics and preaggregate data

In order to simplify joining against weather data, we now preaggregate weather measurements to a single record per weather station and hour.

### Hourly Preaggregation

In [ ]:
hourly_weather = # YOUR CODE HERE

hourly_weather.limit(10).toPandas()

### Daily Preaggregation

In addition to the hourly metrics, we also preaggregate the data to daily records.

In [ ]:
daily_weather = hourly_weather.groupBy("usaf", "wban", "date")\
    .agg(
        f.avg("temperature").alias("temperature"),
        f.avg("wind_speed").alias("wind_speed"),
        f.sum("precipitation").alias("precipitation"),
    )

daily_weather.limit(10).toPandas()

### Save Preaggregated Weather

Finally we save both tables (hourly and daily weather), so we can directly reuse the data in the next steps.

In [ ]:
hourly_weather.write.parquet(refined_basedir + "/weather-hourly/2013")
daily_weather.write.parquet(refined_basedir + "/weather-daily/2013")

## 2.3 Reload Data and draw Pictures

Now let us reload the data (just to make sure everything worked out nicely) and let's draw some pictures. We use a single station (which, by pure incident, is a weather station in NYC)

In [ ]:
daily_weather = spark.read.parquet(refined_basedir + "/weather-daily/2013")

In [ ]:
nyc_station_usaf = "725053"
nyc_station_wban = "94728"

# Filter data only of that weather station, order it by date and convert it to a Pandas DataFrame
pdf = # YOUR CODE HERE

### Wind Speed

The first picture will simply contain the wind speed for every day in 2013.

In [ ]:
# Make a Plot
plt.figure(figsize=(16, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(pdf["date"],pdf["wind_speed"])

### Air Temperature

The next picture contains the average air temperature for every day in 2013.

In [ ]:
# Make a Plot
plt.figure(figsize=(16, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(pdf["date"],pdf["temperature"])

### Precipitation

The last picture contains the precipitation for every day in 2013.

In [ ]:
# Make a Plot
plt.figure(figsize=(16, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(pdf["date"],pdf["precipitation"])